In [10]:
# Definir la tabla de símbolos predefinidos
symbol_table = {
    "SP": 0, "LCL": 1, "ARG": 2, "THIS": 3, "THAT": 4,
    "R0": 0, "R1": 1, "R2": 2, "R3": 3, "R4": 4, "R5": 5, "R6": 6, "R7": 7,
    "R8": 8, "R9": 9, "R10": 10, "R11": 11, "R12": 12, "R13": 13, "R14": 14, "R15": 15,
    "SCREEN": 16384, "KBD": 24576
}

In [11]:
# Leer el archivo de entrada y limpiar comentarios y líneas vacías
def clean_code(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    clean_lines = []
    for line in lines:
        line = line.split('//')[0].strip()  # Elimina comentarios y espacios en blanco
        if line:
            clean_lines.append(line)
    return clean_lines

In [12]:
# Primer pase: Manejo de etiquetas
def first_pass(clean_lines):
    line_number = 0
    for line in clean_lines:
        if line.startswith("("):  # Esto es una etiqueta
            symbol = line[1:-1]  # Eliminar paréntesis
            symbol_table[symbol] = line_number
        else:
            line_number += 1


In [13]:
# Traducción de instrucciones C a binario
def translate_c_instruction(dest, comp, jump):
    # Mapeo de las partes comp, dest y jump
    comp_dict = {
        "0": "0101010", "1": "0111111", "-1": "0111010",
        "D": "0001100", "A": "0110000", "M": "1110000",
        "!D": "0001101", "!A": "0110001", "!M": "1110001",
        "-D": "0001111", "-A": "0110011", "-M": "1110011",
        "D+1": "0011111", "A+1": "0110111", "M+1": "1110111",
        "D-1": "0001110", "A-1": "0110010", "M-1": "1110010",
        "D+A": "0000010", "D+M": "1000010", "D-A": "0010011",
        "D-M": "1010011", "A-D": "0000111", "M-D": "1000111",
        "D&A": "0000000", "D&M": "1000000", "D|A": "0010101",
        "D|M": "1010101"
    }
    dest_dict = {
        "": "000", "M": "001", "D": "010", "MD": "011", "A": "100", "AM": "101", "AD": "110", "AMD": "111"
    }
    jump_dict = {
        "": "000", "JGT": "001", "JEQ": "010", "JGE": "011",
        "JLT": "100", "JNE": "101", "JLE": "110", "JMP": "111"
    }
    comp_bits = comp_dict.get(comp, "0000000")
    dest_bits = dest_dict.get(dest, "000")
    jump_bits = jump_dict.get(jump, "000")

    return f"111{comp_bits}{dest_bits}{jump_bits}"




In [14]:
# Segundo pase: Traducción completa
def second_pass(clean_lines):
    current_variable_address = 16
    binary_lines = []

    for line in clean_lines:
        if line.startswith("@"):  # Instrucción A
            symbol = line[1:]
            if symbol.isdigit():
                address = int(symbol)
            elif symbol in symbol_table:
                address = symbol_table[symbol]
            else:
                symbol_table[symbol] = current_variable_address
                address = current_variable_address
                current_variable_address += 1
            binary_lines.append(f'{address:016b}')

        elif "=" in line or ";" in line:  # Instrucción C
            if "=" in line:
                dest, rest = line.split("=")
            else:
                dest = ""
                rest = line

            if ";" in rest:
                comp, jump = rest.split(";")
            else:
                comp = rest
                jump = ""

            binary_line = translate_c_instruction(dest, comp, jump)
            binary_lines.append(binary_line)

    return binary_lines




In [15]:
# Guardar el archivo de salida
def write_to_file(output_file, binary_lines):
    with open(output_file, 'w') as file:
        for line in binary_lines:
            file.write(line + '\n')


In [16]:
# Función principal para ensamblar el archivo
def assembler(input_file, output_file):
    clean_lines = clean_code(input_file)
    first_pass(clean_lines)
    binary_lines = second_pass(clean_lines)
    write_to_file(output_file, binary_lines)

In [ ]:
from google.colab import files

# Subir archivo .asm desde la computadora
uploaded = files.upload()

# Nombre del archivo ensamblador subido (puede ser "Add.asm", "Max.asm", etc.)
input_file = "Add.asm"
output_file = "output.hack"

# Llamar a la función del ensamblador (el código del ensamblador debe estar definido previamente)
assembler(input_file, output_file)

# Descargar el archivo .hack generado
files.download(output_file)

print(f"Archivo {output_file} generado y descargado automáticamente en la carpeta de descargas.")
